# Jamba 1.5 Family Function Calling Examples

This notebook demonstrates single and multiple function calls to Jamba, showcasing how to extend Large Language Models (LLMs) with external capabilities. By integrating function calling, LLMs can access real-time data, perform specific tasks, and provide more accurate and relevant responses. This approach enhances LLMs' problem-solving abilities and reduces the risk of generating incorrect information.

### Import Required Libraries

In [2]:
!pip install -U "ai21>=2.13.0"

  Using cached sentencepiece-0.2.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (7.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 1.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 8.4 MB/s eta 0:00:00:00:010:01
Using cached sentencepiece-0.2.0-cp311-cp311-macosx_10_9_x86_64.whl (1.2 MB)

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [21]:
# import required libraries
from ai21 import AI21Client
from ai21.logger import set_verbose
from ai21.models.chat import ChatMessage, ToolMessage
from ai21.models.chat.function_tool_definition import FunctionToolDefinition
from ai21.models.chat.tool_defintions import ToolDefinition
from ai21.models.chat.tool_parameters import ToolParameters
import json, requests


# create an instance of the AI21Client
# Sign up for a free account at https://studio.ai21.com/auth
# Once you are signed up, find you API Key at https://studio.ai21.com/account/api-key
client = AI21Client(api_key='',)

### API Access
Both of the examples in this notebook require an Alpha Vantage API key. Keys are free for anyone (for limited use) and can provide up to date financial information for investors and traders programmatically.

In [22]:
# https://www.alphavantage.co/support/#api-key
# You'll need to sign up for a free API key at Alpha Vantage, these are free for limited usage
ALPHA_VANTAGE_API_KEY = 'C4BS9X5EFVM3MZ6X'

### Example 1 | Using Jamba to Call a Single Function

In [23]:
# Define the stock information function
def get_stock_info(symbol: str) -> dict:
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()
   
    # https://www.alphavantage.co/documentation/
    if "Global Quote" in data:
        quote = data["Global Quote"]
        return {
            "symbol": quote.get("01. symbol", "N/A"),
            "price": quote.get("05. price", "N/A"),
            "change": quote.get("09. change", "N/A"),
            "change_percent": quote.get("10. change percent", "N/A"),
            "last_trading_day": quote.get("07. latest trading day", "N/A")
        }
    else:
        return None

In [24]:
# Test the function manually w/o Jamba to verify results
get_stock_info("NVDA")

{'symbol': 'NVDA',
 'price': '121.4000',
 'change': '-2.6400',
 'change_percent': '-2.1283%',
 'last_trading_day': '2024-09-27'}

In [25]:
# Initialize chat messages
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful stock market assistant. Use the supplied tool to retrieve the latest stock information for users. Always include the stock price and the percentage change in your response."
    ),
    ChatMessage(role="user", content="Can you tell me the percentage change for the NVDA stock price over the past 2 months"),
]

In [26]:
# Define the tool for getting stock information
tool_definition = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="get_stock_info",
        description="Get the latest stock information for a given stock symbol",
        parameters=ToolParameters(
            type="object",
            properties={
                "symbol": {"type": "string", "description": "The stock symbol (e.g., AAPL for Apple)"}
            },
            required=["symbol"],
        ),
    ),
)

In [27]:
# Create a list of tools
tools = [tool_definition]

# Make the initial request to the model
response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)

# Process the response and handle tool calls
assistant_message = response.choices[0].message
messages.append(assistant_message)  # Adding the assistant message to the chat history
result = None
tool_calls = assistant_message.tool_calls

if tool_calls:
    tool_call = tool_calls[0]
    if tool_call.function.name == "get_stock_info":
        func_arguments = json.loads(tool_call.function.arguments)
        if "symbol" in func_arguments:
            result = get_stock_info(func_arguments["symbol"])
        else:
            print("Missing 'symbol' in function arguments")
    else:
        print(f"Unexpected tool call found - {tool_call.function.name}")
else:
    print("No tool calls found")

if result is not None:
    # Continue the conversation by passing the stock information result back to the model
    tool_message = ToolMessage(role="tool", tool_call_id=tool_calls[0].id, content=json.dumps(result))
    messages.append(tool_message)
    response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
    print("Final response:")
    print(response.choices[0].message.content)

Final response:
The NVDA stock price is currently at 121.4000. Over the past 2 months, the stock price has changed by -2.6400, which is a percentage change of -2.1283%.


### Example 2 | Using Jamba to call Multiple Functions

In [16]:
# define the function to get the stock information from Alpha Vantage
def get_stock_info(symbol: str) -> dict:
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={ALPHA_VANTAGE_API_KEY}"
    response = requests.get(url)
    data = response.json()
   
    if "Global Quote" in data:
        quote = data["Global Quote"]
        return {
            "symbol": quote.get("01. symbol", "0"),
            "price": float(quote.get("05. price", "0")),
            "change": quote.get("09. change", "N/A"),
            "change_percent": quote.get("10. change percent", "N/A"),
            "last_trading_day": quote.get("07. latest trading day", "N/A")
        }
    else:
        return None

In [17]:
# define the function to calculate the value of the shares
def calculate_share_value(price: float, shares: int) -> float:
    return price * shares

In [18]:
# Define the messages for the chat, the first message is a system message to provide guidance to the user
# The second message is the user message that will be processed by the assistant
messages = [
    ChatMessage(
        role="system",
        content="You are a helpful stock market assistant. Use the supplied tools to assist the user with stock information and calculations. Always use the get_stock_info tool first to retrieve the latest stock price, then use the calculate_share_value tool to determine the total value of shares. Always provide both the stock price and the calculated share value in your final response."
    ),
    ChatMessage(role="user", content="Can you tell me the latest stock price for nVidia and calculate the value of 400 shares?"),
]

In [19]:
# define to Jamba how to handle the api call to get the stock information
get_stock_info_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="get_stock_info",
        description="Get the latest stock information for a given stock symbol",
        parameters=ToolParameters(
            type="object",
            properties={
                "symbol": {"type": "string", "description": "The stock symbol (e.g., AAPL for Apple)"}
            },
            required=["symbol"],
        ),
    ),
)

# define to Jamba how to calculate the share value
calculate_share_value_tool = ToolDefinition(
    type="function",
    function=FunctionToolDefinition(
        name="calculate_share_value",
        description="Calculate the total value of shares by multiplying price times number of shares",
        parameters=ToolParameters(
            type="object",
            properties={
                "price": {"type": "number", "description": "The price per share"},
                "shares": {"type": "integer", "description": "The number of shares"}
            },
            required=["price", "shares"],
        ),
    ),
)

In [34]:
# define the tools to be used
tools = [get_stock_info_tool, calculate_share_value_tool]

# Make the initial request to the model
def process_tool_calls(assistant_message):
    tool_call_id_to_result = {}
    tool_calls = assistant_message.tool_calls
    if tool_calls:
        for tool_call in tool_calls:
            if tool_call.function.name == "get_stock_info":
                func_arguments = json.loads(tool_call.function.arguments)
                if "symbol" in func_arguments:
                    result = get_stock_info(func_arguments["symbol"])
                    tool_call_id_to_result[tool_call.id] = result
                else:
                    print(f"Got unexpected arguments in function call - {func_arguments}")
            elif tool_call.function.name == "calculate_share_value":
                func_arguments = json.loads(tool_call.function.arguments)
                if "price" in func_arguments and "shares" in func_arguments:
                    result = calculate_share_value(func_arguments["price"], func_arguments["shares"])
                    tool_call_id_to_result[tool_call.id] = result
                else:
                    print(f"Got unexpected arguments in function call - {func_arguments}")
            else:
                print(f"Unexpected tool call found - {tool_call.function.name}")
    return tool_call_id_to_result

# Initial response
response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
assistant_message = response.choices[0].message
messages.append(assistant_message)
tool_call_id_to_result = process_tool_calls(assistant_message)

In [ ]:
# OPTIONAL | If both tools weren't used, prompt for completion
# if len(tool_call_id_to_result) < 2:
#     messages.append(ChatMessage(
#         role="user",
#         content="Please make sure to calculate the value of 100 shares using the stock price you retrieved."
#     ))
#     response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
#     assistant_message = response.choices[0].message
#     messages.append(assistant_message)
    
#     additional_results = process_tool_calls(assistant_message)
#     tool_call_id_to_result.update(additional_results)

In [35]:
# Add tool results to messages
for tool_id_called, result in tool_call_id_to_result.items():
    tool_message = ToolMessage(role="tool", tool_call_id=tool_id_called, content=str(result))
    messages.append(tool_message)

# Final response
response = client.chat.completions.create(messages=messages, model="jamba-1.5-large", tools=tools)
final_response = response.choices[0].message.content
final_response

'The latest stock price for nVidia (NVDA) is 123.51. The value of 400 shares is 49404.0.'